## Data Collection

### Sustainable amenities in US cities from OSM

This notebook details the data collection process with functions to:

- Query the Overpass API for all specified nodes (based on key:value tag) in the United States
- For a given city centroid (lat lon), count the nodes that fall into the circular bounding buffer area of specified radius r
- Generate dataset for full list of samples (cities and centroids) and features (OSM amenity types)

In [1]:
import pandas as pd
import numpy as np
import overpy
import matplotlib.pyplot as plt
import math
import time
import requests
import geopandas as gpd
from shapely.geometry import Point, Polygon
from math import radians, cos, sin, asin, sqrt


df_raw = pd.read_csv(r"PATH\uscities.csv")

df = df_raw[['city', 'state_id', 'state_name', 'county_name', 'lat', 'lng', 'population', 'population_proper', 'density', 'incorporated', 'township']]

df['area_sqkm'] = df['population']/df['density']
df['area_sqm'] = df['area_sqkm']/2.59
df['radius_km'] = (df['area_sqkm']/math.pi)**0.5
df['radius_m'] = round(df['radius_km']*1000,0)

<ipython-input-1-a6b12d29e9ae>:13: DtypeWarning: Columns (2,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(r"PATH\uscities.csv")
<ipython-input-1-a6b12d29e9ae>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['area_sqkm'] = df['population']/df['density']
<ipython-input-1-a6b12d29e9ae>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['area_sqm'] = df['area_sqkm']/2.59
<ipython-input-1-a6b12d29e9ae>:19: SettingWithCopyWarning: 
A value is trying to

In [2]:
# Overpass query for amenities

def query_amenities(amenity):
    result = {}
    start_time = time.time()
    api = overpy.Overpass(max_retry_count=None, retry_timeout=5)
    r = api.query(f"""
    [out:json][timeout:360];
    (area["ISO3166-1"="US"];
    node(area)["amenity"="{amenity}"];
    );
    out;
    """)

    print("query time: ", str(time.time() - start_time))

    # response
    return r.nodes

def query_leisure(leisure):
    result = {}
    start_time = time.time()
    api = overpy.Overpass(max_retry_count=None, retry_timeout=5)
    r = api.query(f"""
    [out:json][timeout:360];
    (area["ISO3166-1"="US"];
    node(area)["leisure"="{leisure}"];
    );
    out;
    """)

    print("query time: ", str(time.time() - start_time))

    # response
    return r.nodes

#https://stackoverflow.com/questions/42686300/how-to-check-if-coordinate-inside-certain-area-python
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

# helper
def count_amenities(lat, lon, rad, osm_query):
    "lat, lon for centroid"
    counter = 0
    
    for i in osm_query:
        lat_ = float(i.lat)
        lon_ = float(i.lon)

        ## Distance of point i to centroid
        a = haversine(lon, lat, lon_, lat_)
        
        if a <= rad:
            counter += 1
        else:
            continue
    
    return counter

# take OSM query response and produce df of cities with counts of amenities within radius of centroid
def map_to_cities(df, amenity, leisure=False):
    "df = city dataframe" 
    
    r = query_amenities(amenity)
    
    df[str(amenity+'_count')] = df.apply(lambda row: count_amenities(row['lat'], row['lng'], row['radius_km'], osm_query=r), axis=1)
    
    return df

def gen_df(cities_df, amenities, leisure=False):
    "take input cities_df and generate output with amenity counts"
    
    output = cities_df
    
    for i in amenities:
        map_to_cities(output, amenity=i, leisure=leisure)
        time.sleep(60)
        
    return output

Reading TagInfo

In [3]:
import bz2
import sqlite3
import pandas as pd
import sys
import numpy as np

# TagInfo master
file = r"PATH\TagInfo\taginfo-master.db\taginfo-master.db"
cnx = sqlite3.connect(file)
db = pd.read_sql_query("SELECT * FROM sqlite_master", cnx)

# TagInfo wiki
file_w = r"PATH\TagInfo\taginfo-wiki.db\taginfo-wiki.db"
cnx_w = sqlite3.connect(file_w)
db_wiki = pd.read_sql_query("SELECT * FROM sqlite_master", cnx_w)

master_stats = pd.read_sql_query("SELECT * FROM master_stats", cnx)
top_tags = pd.read_sql_query("SELECT * FROM top_tags", cnx) # key value pairs
popular_keys = pd.read_sql_query("SELECT * FROM popular_keys", cnx) # top keys
popular_metadata = pd.read_sql_query("SELECT * FROM popular_metadata", cnx)

In [4]:
## amenity key-value pairs with highest count of nodes tagged
top_tags['key-value'] = top_tags['skey'] + top_tags['svalue']
top_tags[top_tags['skey']=='amenity'].sort_values('count_nodes', ascending=False)[['skey', 'svalue', 'count_nodes']]
values = ['language_school', 'library', 'public_bookcase', 'childcare','social_facility','community_centre','waste_basket','waste_disposal','recycling','bicycle_repair_station','bus_station','charging_point', 'bicycle_parking']
result = top_tags[(top_tags['skey']=='amenity') & (top_tags['svalue'].isin(values))]
result.sort_values('count_nodes')

,skey,svalue,count_all,count_nodes,count_ways,count_relations,in_wiki,in_wiki_en,projects,key-value
4753,amenity,public_bookcase,13921,13601,319,1,1,1,10.0,amenitypublic_bookcase
4710,amenity,childcare,40429,20874,19387,168,1,1,7.0,amenitychildcare
4705,amenity,bus_station,61652,37241,23714,697,1,1,15.0,amenitybus_station
4715,amenity,community_centre,142834,57534,84155,1145,1,1,13.0,amenitycommunity_centre
4735,amenity,library,92698,59023,32896,779,1,1,17.0,amenitylibrary
4760,amenity,social_facility,122695,60926,58455,3314,1,1,11.0,amenitysocial_facility
4771,amenity,waste_disposal,165389,150498,14837,54,1,1,7.0,amenitywaste_disposal
4755,amenity,recycling,348141,322969,24900,272,1,1,19.0,amenityrecycling
4701,amenity,bicycle_parking,448613,380509,67865,239,1,1,15.0,amenitybicycle_parking
4770,amenity,waste_basket,571259,570805,448,6,1,1,14.0,amenitywaste_basket


#### Unit tests

In [5]:
r = query_amenities('public_bookcase')

query time:  37.653977155685425


In [6]:
r = query_leisure('park')

query time:  95.49194264411926


In [8]:
r = query_leisure('fitness_station')

sample = df.head(200)

sample[str('fitness_station')] = sample.apply(lambda row: count_amenities(row['lat'], row['lng'], row['radius_km'], osm_query=r), axis=1)

query time:  23.790497303009033


<ipython-input-8-3823fbdc71d4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample[str('fitness_station')] = sample.apply(lambda row: count_amenities(row['lat'], row['lng'], row['radius_km'], osm_query=r), axis=1)


In [9]:
sample.sort_values('fitness_station')

,city,state_id,state_name,county_name,lat,lng,population,population_proper,density,incorporated,township,area_sqkm,area_sqm,radius_km,radius_m,fitness_station
99,Reno,NV,Nevada,Washoe,39.5497,-119.8483,445020.0,255601.0,907.0,True,False,490.650496,189.440346,12.497156,12497.0,0
119,Modesto,CA,California,Stanislaus,37.6374,-121.0028,383134.0,215196.0,1933.0,True,False,198.206932,76.527773,7.942999,7943.0,0
121,Aurora,CO,Colorado,Adams,39.7087,-104.7273,379289.0,379289.0,949.0,True,False,399.672287,154.313624,11.279168,11279.0,0
123,Fayetteville,AR,Arkansas,Washington,36.0713,-94.1660,351246.0,87590.0,625.0,True,False,561.993600,216.985946,13.374906,13375.0,0
124,Anaheim,CA,California,Orange,33.8390,-117.8572,350365.0,350365.0,2688.0,True,False,130.344122,50.325916,6.441259,6441.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,San Diego,CA,California,San Diego,32.8312,-117.1225,3220118.0,1423851.0,1686.0,True,False,1909.915777,737.419219,24.656542,24657.0,49
3,Miami,FL,Florida,Miami-Dade,25.7839,-80.2102,6445545.0,467963.0,5019.0,True,False,1284.228930,495.841286,20.218377,20218.0,49
1,Los Angeles,CA,California,Los Angeles,34.1139,-118.4068,12750807.0,3979576.0,3276.0,True,False,3892.187729,1502.775185,35.198321,35198.0,56
10,Phoenix,AZ,Arizona,Maricopa,33.5722,-112.0891,4219697.0,1680992.0,1253.0,True,False,3367.675180,1300.260687,32.740866,32741.0,159


#### Dataset

**education**
- amenity	language_school
- amenity	library
- namenity	public_bookcase

**social mobility and cohesion**
- amenity	childcare
- amenity	social_facility
- amenity	community_centre

**clean environment**
- amenity	waste_basket
- amenity	waste_disposal
- amenity	recycling

**green transport**
- amenity	bicycle_repair_station
- amenity	bus_station
- amenity charging_point
- amenity	bicycle_parking

**health and wellbeing**
- leisure	park
- leisure	fitness_station

In [ ]:
sdg_amenities = ['recycling',
 'social_centre',
 'bicycle_repair_station',
 'language_school',
 'waste_basket',
 'bicycle_parking',
 'waste_disposal',
 'charging_station',
 'public_bookcase',
 'library',
 'social_facility',
 'community_centre',
 'bus_station',
 'childcare']

sdg_leisures = ['park', 'fitness_station']

**GENERATING DF**

In [ ]:
sample = df.head(200)

df_output = gen_df(cities_df=sample, amenities=sdg_amenities)

# df_output = gen_df(cities_df=sample, amenities=sdg_leisures, leisure=True)